<a href="https://colab.research.google.com/github/sohv/iisc-assignment/blob/main/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install torch numpy tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR
import numpy as np
from tqdm import tqdm
import math

In [ ]:
# ==================== configuration ====================
class Config:
    img_size = 32
    patch_size = 4  # best for CIFAR-10 (32x32 images)
    num_patches = (img_size // patch_size) ** 2  # 64 patches
    in_channels = 3
    embed_dim = 384  # increased from typical 256
    depth = 12
    num_heads = 12
    mlp_ratio = 4
    dropout = 0.1
    attn_dropout = 0.1
    num_classes = 10

    # training Hyperparameters
    batch_size = 128
    epochs = 300
    learning_rate = 3e-4
    weight_decay = 0.05
    warmup_epochs = 10
    label_smoothing = 0.1

    # regularization
    stochastic_depth = 0.1  # drop path
    grad_clip = 1.0

    # device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# ==================== data augmentation ====================
def get_transforms(train=True):
    if train:
        return transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                               (0.2470, 0.2435, 0.2616)),
            transforms.RandomErasing(p=0.25, scale=(0.02, 0.33)),
        ])
    else:
        return transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                               (0.2470, 0.2435, 0.2616)),
        ])

In [ ]:
# ==================== patch embedding ====================
class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, embed_dim):
        super().__init__()
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_size = patch_size

        self.projection = nn.Conv2d(
            in_channels, embed_dim,
            kernel_size=patch_size, stride=patch_size
        )

    def forward(self, x):
        # x: (B, C, H, W) -> (B, E, H/P, W/P) -> (B, E, N) -> (B, N, E)
        x = self.projection(x)  # (B, embed_dim, H/P, W/P)
        x = x.flatten(2)  # (B, embed_dim, num_patches)
        x = x.transpose(1, 2)  # (B, num_patches, embed_dim)
        return x

In [ ]:
# ==================== stochastic depth - drop path ====================
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.0):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        output = x.div(keep_prob) * random_tensor
        return output

In [ ]:
# ==================== multi-head self-attention ====================
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, attn_dropout=0.0, proj_dropout=0.0):
        super().__init__()
        assert embed_dim % num_heads == 0

        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(embed_dim, embed_dim * 3, bias=True)
        self.attn_dropout = nn.Dropout(attn_dropout)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.proj_dropout = nn.Dropout(proj_dropout)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # scaled dot-product attention
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = F.softmax(attn, dim=-1)
        attn = self.attn_dropout(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_dropout(x)
        return x

In [ ]:
# ==================== MLP block ====================
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, dropout=0.0):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, in_features)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

In [ ]:
# ==================== Transformer encoder block ====================
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4, dropout=0.0,
                 attn_dropout=0.0, drop_path=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadSelfAttention(
            embed_dim, num_heads, attn_dropout, dropout
        )
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = MLP(embed_dim, int(embed_dim * mlp_ratio), dropout)

    def forward(self, x):
        # pre norm architecture
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

In [ ]:
# ==================== Define vision transformer ====================
class VisionTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # patch embedding
        self.patch_embed = PatchEmbedding(
            config.img_size, config.patch_size,
            config.in_channels, config.embed_dim
        )

        # CLS token and positional embeddings
        self.cls_token = nn.Parameter(torch.zeros(1, 1, config.embed_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, config.num_patches + 1, config.embed_dim)
        )
        self.pos_dropout = nn.Dropout(config.dropout)

        # stochastic depth decay rule
        dpr = [x.item() for x in torch.linspace(0, config.stochastic_depth, config.depth)]

        # Transformer encoder blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(
                config.embed_dim, config.num_heads, config.mlp_ratio,
                config.dropout, config.attn_dropout, dpr[i]
            ) for i in range(config.depth)
        ])

        # classification head
        self.norm = nn.LayerNorm(config.embed_dim)
        self.head = nn.Linear(config.embed_dim, config.num_classes)

        # initializing weights
        self._init_weights()

    def _init_weights(self):
        # initialize patch embedding
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)

        # initialize linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.LayerNorm):
                nn.init.constant_(m.bias, 0)
                nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        B = x.shape[0]

        # patch embedding
        x = self.patch_embed(x)  # (B, num_patches, embed_dim)

        # prepend CLS token
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)  # (B, num_patches+1, embed_dim)

        # add positional embeddings
        x = x + self.pos_embed
        x = self.pos_dropout(x)

        # transformer encoder blocks
        for block in self.blocks:
            x = block(x)

        # final norm and classification head
        x = self.norm(x)
        x = x[:, 0]  # extracting CLS token
        x = self.head(x)

        return x

In [ ]:
# training functions
def train_epoch(model, loader, criterion, optimizer, scheduler, config, scaler):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc='Training')
    for inputs, targets in pbar:
        inputs, targets = inputs.to(config.device), targets.to(config.device)

        optimizer.zero_grad()

        # Mixed precision training
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
        scaler.step(optimizer)
        scaler.update()

        if scheduler is not None and isinstance(scheduler, OneCycleLR):
            scheduler.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })

    return total_loss / len(loader), 100. * correct / total

In [ ]:
def validate(model, loader, criterion, config):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in tqdm(loader, desc='Validating'):
            inputs, targets = inputs.to(config.device), targets.to(config.device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total

In [ ]:
# ====================== Main training function =====================
def main():
    config = Config()
    print(f"Using device: {config.device}")
    print(f"Model configuration: {config.embed_dim}D, {config.depth} layers, {config.num_heads} heads")

    # Load data
    print("Loading CIFAR-10 dataset...")
    train_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True,
        transform=get_transforms(train=True)
    )
    test_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True,
        transform=get_transforms(train=False)
    )

    train_loader = DataLoader(
        train_dataset, batch_size=config.batch_size,
        shuffle=True, num_workers=2, pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=2, pin_memory=True
    )

    # create model
    print("Creating Vision Transformer model...")
    model = VisionTransformer(config).to(config.device)

    # count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

    # Loss function with label smoothing
    criterion = nn.CrossEntropyLoss(label_smoothing=config.label_smoothing)

    # Optimizer (AdamW)
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=config.learning_rate,
        weight_decay=config.weight_decay, betas=(0.9, 0.999)
    )

    # Learning rate scheduler
    scheduler = CosineAnnealingLR(optimizer, T_max=config.epochs, eta_min=1e-6)

    # Mixed precision scaler
    scaler = torch.cuda.amp.GradScaler()

    # Training loop
    best_acc = 0
    print(f"\nStarting training for {config.epochs} epochs...")

    for epoch in range(config.epochs):
        print(f"\nEpoch {epoch+1}/{config.epochs}")

        # Warmup
        if epoch < config.warmup_epochs:
            warmup_lr = config.learning_rate * (epoch + 1) / config.warmup_epochs
            for param_group in optimizer.param_groups:
                param_group['lr'] = warmup_lr

        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, scheduler, config, scaler
        )
        test_loss, test_acc = validate(model, test_loader, criterion, config)

        if epoch >= config.warmup_epochs:
            scheduler.step()

        current_lr = optimizer.param_groups[0]['lr']
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
        print(f"Learning Rate: {current_lr:.6f}")

        # Save best model
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'test_acc': test_acc,
                'config': config
            }, 'best_vit_cifar10.pth')
            print(f"✓ Saved best model with test accuracy: {best_acc:.2f}%")

    print(f"\nTraining completed! Best test accuracy: {best_acc:.2f}%")
    return best_acc

if __name__ == '__main__':
    main()

Using device: cuda
Model configuration: 384D, 12 layers, 12 heads
Loading CIFAR-10 dataset...
Creating Vision Transformer model...


/tmp/ipykernel_619/880310869.py:50: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Total parameters: 21,342,346
Trainable parameters: 21,342,346

Starting training for 300 epochs...

Epoch 1/300


Training:   0%|          | 0/391 [00:00<?, ?it/s]/tmp/ipykernel_619/3995718836.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating: 100%|██████████| 79/79 [00:02<00:00, 33.14it/s]


Train Loss: 2.1261, Train Acc: 21.86%
Test Loss: 1.9134, Test Acc: 32.85%
Learning Rate: 0.000030
✓ Saved best model with test accuracy: 32.85%

Epoch 2/300


...

Train Loss: 0.6059, Train Acc: 95.28%
Test Loss: 0.7353, Test Acc: 92.07%
Learning Rate: 0.000002

Epoch 299/300


Validating: 100%|██████████| 79/79 [00:02<00:00, 33.25it/s]


Train Loss: 0.6041, Train Acc: 95.42%
Test Loss: 0.7360, Test Acc: 92.17%
Learning Rate: 0.000002

Epoch 300/300


Validating: 100%|██████████| 79/79 [00:02<00:00, 33.14it/s]

Train Loss: 0.6031, Train Acc: 95.42%
Test Loss: 0.7351, Test Acc: 92.11%
Learning Rate: 0.000002

Training completed! Best test accuracy: 92.20%
